In [42]:
import ctypes
import os

# Install CANape. Not included for copyright reasons.
header_file = r"C:\Program Files\Vector CANape 17\CANapeAPI\CANapAPI.h"
assert os.path.exists(header_file)
CANapAPI_dll= r"C:\Program Files\Vector CANape 17\CANapeAPI\CANapAPI64.dll"
assert os.path.exists(CANapAPI_dll)

# Load the library and assign it to dll.
dll = ctypes.windll.LoadLibrary(CANapAPI_dll)
# Define version_t as a Python ctypes-structure.
class version_t(ctypes.Structure):
    _fields_ = [
        ('dllMainVersion', ctypes.c_int),
        ('dllSubVersion', ctypes.c_int),
        ('dllRelease', ctypes.c_int),
        ('osVersion', ctypes.c_char * 50),
        ('osRelease', ctypes.c_int),
    ]
    
    def __eq__(self, other):
        if isinstance(other, type(self)):
            return str(other)==str(self)
        if isinstance(other, str):
            return str(other)==str(self)
        raise Exception(f"Unsupported class comparison {type(other)}")
    
    def __repr__(self):
        return f"API_VERSION<{self.dllMainVersion}.{self.dllSubVersion}.{self.dllRelease}>"

    def __str__(self):
        return "{}.{}.{}".format(self.dllMainVersion, self.dllSubVersion, self.dllRelease)

## Set the argument and return types.
# Pass by reference.
dll.Asap3GetVersion.argtypes = (ctypes.POINTER(version_t),)
# Return a success bool.
dll.Asap3GetVersion.restype = ctypes.c_bool

#def get_version():
version = version_t()
result = dll.Asap3GetVersion(ctypes.byref(version))
assert version==version
assert version=="2.3.1"

In [10]:
dll

<WinDLL 'C:\Program Files\Vector CANape 17\CANapeAPI\CANapAPI64.dll', handle 7ffdb37f0000 at 0x28abd9010f0>

In [11]:
result, version

(True, API_VERSION<2.3.1>)

In [12]:
assert version==version

assert version=="2.3.1"

In [13]:
assert version.osRelease==0
assert version.osVersion==b'Windows95/WindowsNT'

version.osVersion, version.osRelease

(b'Windows95/WindowsNT', 0)

In [14]:
version.dllMainVersion, version.dllSubVersion, version.dllRelease

(2, 3, 1)

# Open CANape in Modal Mode

Modal mode is the best way I've found to reverse engineer this. It allows you to relequish control from Python and do something with the mouse.t

In [37]:
class tAsap3Hdl(ctypes.Structure):
    def __repr__(self):
        return "tAsap3Hdl<>"

TAsap3Hdl = ctypes.POINTER(struct_tAsap3Hdl) # C:\\Program Files (x86)\\Vector CANape 14\\CANapeAPI\\CANapAPI.h: 623

def __repr__(self):
    return "TAsap3Hdl<>"
def byref(self):
    return ctypes.byref(self)
setattr(TAsap3Hdl, "__repr__", __repr__)
setattr(TAsap3Hdl, "byref", property(byref))

handle=TAsap3Hdl()

In [38]:
handle

TAsap3Hdl<>

In [39]:
# CANape requires absolute path.
workingDir = os.path.abspath("canape_tmp")
# Maximum response time (ms)
responseTimeout = 10000 # 10 seconds
fifoSize = 8192
sampleSize = 256
debugMode = True
# Clear CANape device list.
clearDeviceList = True
# Start CANape in hex mode
bHexmode = False
# Start CANape in modal mode. (NonModal = True)
bModalMode = False

# Convert to ctypes.
c_responseTimeout = ctypes.c_ulong()
c_workingDir = ctypes.c_char_p(workingDir.encode("UTF-8"))
c_fifoSize = ctypes.c_ulong(fifoSize)
c_sampleSize = ctypes.c_ulong(sampleSize)
c_debugMode = ctypes.c_bool(debugMode)
c_clearDeviceList = ctypes.c_bool(clearDeviceList)
c_bHexmode = ctypes.c_bool(bHexmode)
c_bModalMode = ctypes.c_bool(bModalMode)

In [40]:
dll.Asap3Init5.restype = ctypes.c_bool
dll.Asap3Init5.argtypes = (
    ctypes.POINTER(TAsap3Hdl),
    ctypes.c_ulong,
    ctypes.c_char_p,
    ctypes.c_ulong,
    ctypes.c_ulong,
    ctypes.c_bool,
    ctypes.c_bool,
    ctypes.c_bool,
    ctypes.c_bool
)

In [41]:
result = dll.Asap3Init5(
    handle.byref,
    c_responseTimeout,
    c_workingDir,
    c_fifoSize,
    c_sampleSize,
    c_debugMode,
    c_clearDeviceList,
    c_bHexmode,
    c_bModalMode,
)
result

False

CANape should launch in modal mode and look like this:

![](init5_result.png)

Exit CANape by passing the handle into the exit function.

In [48]:
first_occurance("Asap3Exit")
dll.Asap3Exit.argtypes= (TAsap3Hdl, )
dll.Asap3Exit.restype=ctypes.c_bool

result = dll.Asap3Exit(handle)
result

'Asap3Exit' first occurs on line 84 in C:\Program Files\Vector CANape 17\CANapeAPI\CANapAPI.h


False

Non-Modal Mode ```bModalMode=True```

In [ ]:
result = dll.Asap3Init5(
    handle_p,
    responseTimeout,
    workingDir.encode("UTF-8"),
    fifoSize,
    sampleSize,
    debugMode,
    clearDeviceList,
    bHexmode,
    True
)

CANape default Window after non-modal launch.

![non-modal](init5_result_nonmodal.png)

In [ ]:
first_occurance("Asap3Exit")
dll.Asap3Exit.argtypes= (TAsap3Hdl, )
dll.Asap3Exit.restype=ctypes.c_bool

result = dll.Asap3Exit(handle)
result

# Appendices & Code

Required Software Installed:

- Python 3.x
- Jupyter Notebook
- [Vector CANape 17.0 Demo](https://www.vector.com/int/en/download/?tx_vectorproducts_productdownloaddetail%5Bdownload%5D=43948&tx_vectorproducts_productdownloaddetail%5Baction%5D=show&tx_vectorproducts_productdownloaddetail%5Bcontroller%5D=Productdownload&cHash=3a39f97388a3ea8f6cb6e9371ea6938b) [MD5 hash : 	5e44feeca4aab18b64ecbc556160f66e]